In [136]:
from numpy.random import seed
seed(1)
import sklearn
sklearn.__version__
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import re
from sklearn.metrics import classification_report

In [137]:
import csv

input_file = './data/weather_data_2020.txt'
output_file = './data/weather_data_2020.csv'

with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
    # Read the header and remove leading/trailing whitespaces
    header = [col.strip() for col in txt_file.readline().split()]

    # Initialize the CSV writer with a tab as the delimiter
    csv_writer = csv.writer(csv_file, delimiter='\t')

    # Write the header to the CSV file
    csv_writer.writerow(header)

    # Read and write each line
    for line in txt_file:
        row = [col.strip() for col in line.split('\t')]
        csv_writer.writerow(row)

print(f'Conversion completed. CSV file saved as {output_file}')

Conversion completed. CSV file saved as ./data/weather_data_2020.csv


In [111]:
input_file = './data/weather_data_2021.txt'
output_file = './data/weather_data_2021.csv'

with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
    # Read the header and remove leading/trailing whitespaces
    header = [col.strip() for col in txt_file.readline().split()]

    # Initialize the CSV writer with a tab as the delimiter
    csv_writer = csv.writer(csv_file, delimiter='\t')

    # Write the header to the CSV file
    csv_writer.writerow(header)

    # Read and write each line
    for line in txt_file:
        row = [col.strip() for col in line.split('\t')]
        csv_writer.writerow(row)

print(f'Conversion completed. CSV file saved as {output_file}')

Conversion completed. CSV file saved as ./data/weather_data_2021.csv


In [112]:
input_file = './data/weather_data_2022.txt'
output_file = './data/weather_data_2022.csv'

with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
    # Read the header and remove leading/trailing whitespaces
    header = [col.strip() for col in txt_file.readline().split()]

    # Initialize the CSV writer with a tab as the delimiter
    csv_writer = csv.writer(csv_file, delimiter='\t')

    # Write the header to the CSV file
    csv_writer.writerow(header)

    # Read and write each line
    for line in txt_file:
        row = [col.strip() for col in line.split('\t')]
        csv_writer.writerow(row)

print(f'Conversion completed. CSV file saved as {output_file}')

Conversion completed. CSV file saved as ./data/weather_data_2022.csv


In [113]:
input_file = './data/weather_data_2023.txt'
output_file = './data/weather_data_2023.csv'

with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
    # Read the header and remove leading/trailing whitespaces
    header = [col.strip() for col in txt_file.readline().split()]

    # Initialize the CSV writer with a tab as the delimiter
    csv_writer = csv.writer(csv_file, delimiter='\t')

    # Write the header to the CSV file
    csv_writer.writerow(header)

    # Read and write each line
    for line in txt_file:
        row = [col.strip() for col in line.split('\t')]
        csv_writer.writerow(row)

print(f'Conversion completed. CSV file saved as {output_file}')

Conversion completed. CSV file saved as ./data/weather_data_2023.csv


In [114]:
data2020 = pd.read_csv('./data/weather_data_2020.csv', sep="\t")
data2021 = pd.read_csv('./data/weather_data_2021.csv', sep="\t")
data2022 = pd.read_csv('./data/weather_data_2022.csv', sep="\t")
data2023 = pd.read_csv('./data/weather_data_2023.csv', sep="\t")
data = pd.concat([data2020,data2021,data2022,data2023], ignore_index=True, axis=0)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11552 entries, 0 to 11551
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         11552 non-null  object 
 1   month        11552 non-null  int64  
 2   temperature  11552 non-null  int64  
 3   feelslike    11552 non-null  int64  
 4   wind         11552 non-null  int64  
 5   direction    11552 non-null  object 
 6   gust         11552 non-null  int64  
 7   cloud        11552 non-null  int64  
 8   rain         11552 non-null  float64
 9   rain_ratio   11552 non-null  int64  
 10  pressure     11552 non-null  int64  
 11  weather      11552 non-null  object 
dtypes: float64(1), int64(8), object(3)
memory usage: 1.1+ MB


In [115]:
# Xếp loại các loại thời tiết thành 2 dạng là có mưa và không mưa
list_norain = ['Clear', 'Fog', 'Cloudy', 'Mist', 'Partly cloudy', 'Thundery outbreaks possible']
list_rain = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain', 
             'Patchy light rain with thunder','Patchy rain possible','Heavy rain','Heavy rain at times',
             'Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast','Torrential rain shower']

In [116]:
# Lấy ra cột y để train
# Cột y này là giá trị thời tiết được xếp loại
y = []
for w in data.weather:
    if w in list_norain:
        y+= ['norain']
    elif w in list_rain:
        y+= ['rain']
    else:
        y+= ['unknown']
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo
# nên chúng ta xóa dòng đầu tiên
y.pop(0)
print(y)

['rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'norain', 'rain', 'rain', 'norain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'norain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'rain', 'rain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'rain', 'norain', 'norain', 'norain', 'norain', 'rain', 'norain', 'norain', 'rain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'rain', 'rain', 'rain', 'rain', 'rain', 'norain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'norain', 'rain', 'rain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'norain', 'rain', 'rain', 'rain', 'rain',

In [117]:
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo nhưng không có giá trị đó nên xóa dòng cuối cùng
x = data.drop(11551)


In [118]:
# Tách tập train và tập test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify = y, random_state=0)

In [119]:
# Tách tập train và tập validation
x_train_v, x_val, y_train_v, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state=0)

In [120]:
# Góm nhóm các giá trị của các cột có kiểu category
# Cột month gom thành 2 giá trị gồm "mua" (5->11) và "kho" (12->4)
# Cột direction thì xóa bỏ kí tự đầu tiên nếu vượt quá 2 ký tự
# Cột weather thì gom nhóm giống như khi gom nhóm y
class ColStd(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df,y=None):
        df = X_df[['time',
                   'month',
                   'temperature',
                   'feelslike',
                   'wind',
                   'direction',
                   'gust',
                   'cloud',
                   'rain',
                   'pressure',
                   'weather']]#khong lay cot precipitatinon
        temp_time =[]
        for x in df['time']:
            t = int(re.findall(r'(\d{1,2}):\d{2}',x)[0])
            if ((t>=6) & (t<=15)): 
                temp_time+=['sang']
            else:
                temp_time+=['toi']
        df.time=temp_time
        temp_month =[]
        for x in df['month']:
            if x in [5,6,7,8,9,10,11]:
                temp_month += ['mua'] 
            elif x in [12,1,2,3,4]:
                temp_month += ['kho']
            else:
                temp_month += ['kho']
        df.month = temp_month
        temp_dic = []
        for x in df['direction']:
            if len(x) == 3:
                temp_dic += [x[1:]]
            else:
                temp_dic += [x]
        df.direction = temp_dic
        temp_wea = []
        for w in df['weather']:
            if w in list_norain:
                temp_wea+= ['norain']
            #elif w in list_ltrain:
            #    temp_wea+= ['ltrain']
            elif w in list_rain:
                temp_wea+= ['rain']
            else:
                temp_wea+= ['unknown']
        df.weather = temp_wea
        return df

In [121]:
# Tạo các Pipeline để tiền xử lý các giá trị có trong dữ liệu
nume_cols = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'rain', 'pressure']
#nume_cols_nan = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity','precipitation', 'pressure']
cate_cols = ['time', 'month', 'direction', 'weather']

nume_trans = Pipeline(steps=[
    ('imputer1', SimpleImputer(missing_values = np.nan, strategy = 'mean')),
    ('imputer2', SimpleImputer(missing_values = 0, strategy='mean'))
])

cate_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('onehot', OneHotEncoder(dtype=int, handle_unknown= 'ignore'))
])

cols_trans = ColumnTransformer(
    transformers=[
        ('nume', nume_trans, nume_cols),
        ('cate', cate_trans, cate_cols)
])

preprocess_pipeline = Pipeline(steps=[
    ('colstd', ColStd()),
    ('coltrans', cols_trans),
    ('std', StandardScaler())
])

# Tiền xử lý tập train x
preprocessed_train_X = preprocess_pipeline.fit_transform(x_train_v)

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [122]:
# Tiền xử lý tập validation x
preprocess_val_X = preprocess_pipeline.transform(x_val)

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [123]:
from sklearn.utils.class_weight import compute_class_weight 

In [124]:
#khảo sát số lượng từng nhãn xem có cân bằng hay không
rain = 0
norain =0
for y in y_train:
    if y =='rain':
        rain+=1
    else:
        norain+=1
print(f"rain: {rain} norain: {norain}")

rain: 4816 norain: 5579


In [125]:
# Tạo ra full_pipeline bao gồm tiền xử lý và model
mlpclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('mlpclf', MLPClassifier(hidden_layer_sizes=(20),activation='tanh', solver='lbfgs', random_state=0, max_iter=500,))
])
#hidden_layer_sizes = (10, 20, 50, 100)

In [126]:
# train và tìm ra các tham số tốt nhất
train_errs = []
val_errs = []
iter = []
alphas = [0.001,0.01, 0.1, 1, 10,20, 100]

hidden_layer_sizes = [1,10,20,50,70,100]
best_val_err = float('inf'); best_alpha = None; best_hls = None;
for alpha in alphas:
    for hls in hidden_layer_sizes:
        iter +=[f"alpha = {alpha}, hls = {hls}"]
        print(iter[-1])
        mlpclf_model.set_params(mlpclf__alpha=alpha,mlpclf__hidden_layer_sizes=(hls))
        mlpclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - mlpclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - mlpclf_model.score(x_val,y_val))

        train_errs += [train_err]
        val_errs +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_alpha = alpha
            best_hls = hls


alpha = 0.001, hls = 1


C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

alpha = 0.001, hls = 10


C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

...

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

alpha = 100, hls = 70


C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

alpha = 100, hls = 100


C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [127]:
for i in range(len(train_errs)):
    print(f"{iter[i]} \n train_err :{train_errs[i]}\t val_err:{val_errs[i]}")
    print()
print(f"best alpha: {best_alpha}")
print(f"best hidden layer size: {best_hls}")

alpha = 0.001, hls = 1 
 train_err :22.739297739297736	 val_err:21.885521885521886

alpha = 0.001, hls = 10 
 train_err :20.791245791245792	 val_err:21.452621452621457

alpha = 0.001, hls = 20 
 train_err :17.989417989417987	 val_err:23.424723424723425

alpha = 0.001, hls = 50 
 train_err :14.694564694564693	 val_err:25.877825877825877

alpha = 0.001, hls = 70 
 train_err :12.31361231361231	 val_err:26.310726310726306

alpha = 0.001, hls = 100 
 train_err :9.896584896584892	 val_err:28.234728234728234

alpha = 0.01, hls = 1 
 train_err :22.72727272727273	 val_err:21.885521885521886

alpha = 0.01, hls = 10 
 train_err :20.538720538720533	 val_err:20.731120731120733

alpha = 0.01, hls = 20 
 train_err :18.193843193843197	 val_err:23.088023088023093

alpha = 0.01, hls = 50 
 train_err :14.65848965848966	 val_err:25.25252525252525

alpha = 0.01, hls = 70 
 train_err :12.590187590187584	 val_err:27.705627705627712

alpha = 0.01, hls = 100 
 train_err :10.365560365560366	 val_err:27.60942760

In [128]:
for i in range(len(train_errs)):
    if (((i+1)%6==0) & (i!=0)):
        print(f"{round(train_errs[i],7)}",end ="\n")
    else:
        print(f"{round(train_errs[i],7)}",end ="\t")

print()
for i in range(len(val_errs)):
    if (((i+1)%6==0 & (i!=0))):
        print(f"{round(val_errs[i],7)}",end ="\n")
    else:
        print(f"{round(val_errs[i],7)}",end ="\t")

22.7392977	20.7912458	17.989418	14.6945647	12.3136123	9.8965849
22.7272727	20.5387205	18.1938432	14.6584897	12.5901876	10.3655604
22.7392977	20.4304954	18.8672439	14.8148148	12.7585378	10.6301106
22.6791727	20.7070707	19.1317941	15.8249158	14.3097643	13.0591631
22.5829726	21.1279461	20.5266955	20.0697451	20.1659452	20.2380952
22.5829726	21.2121212	21.2241462	21.3443963	21.3924964	21.3443963
22.5589226	22.3063973	22.3063973	22.3304473	22.3665224	22.3424723

21.8855219	21.4526215	23.4247234	25.8778259	26.3107263	28.2347282
21.8855219	20.7311207	23.0880231	25.2525253	27.7056277	27.6094276
21.7893218	20.3463203	22.2703223	25.6373256	26.1664262	27.4170274
21.7412217	21.0678211	21.7893218	24.6272246	23.9057239	25.012025
21.4526215	20.7792208	21.1640212	20.4425204	20.6349206	20.8754209
21.3564214	21.1640212	20.7311207	21.0678211	20.6830207	20.8273208
21.4045214	21.1640212	21.1640212	21.1640212	21.2602213	21.2602213


In [129]:
svmclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('svm', SVC(kernel = 'rbf'))
])
c = [0.1,1,10, 20, 50,100]
gamma = [0.0001,0.001,0.01,0.1,1]
train_errs1 = []
val_errs1 = []
iter = []
best_val_err = float('inf'); best_C = None; best_gamma=None
for C in c:
    for g in gamma:
        iter +=[f"C={C}, g={g}"]
        print(iter[-1])
        svmclf_model.set_params(svm__C=C,svm__gamma=g)
        svmclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - svmclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - svmclf_model.score(x_val,y_val))
        train_errs1 += [train_err]
        val_errs1 +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_C = C
            best_gamma =g
    

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

C=0.1, g=0.0001


C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

C=0.1, g=0.001


C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

...

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

C=100, g=0.1


C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

C=100, g=1


C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [130]:
for i in range(len(train_errs1)):
    print(f"{iter[i]} \n train_err :{train_errs1[i]}\t val_err:{val_errs1[i]}")
    print()
print(f"best C: {best_C}")
print(f"best gamma: {best_gamma}")

C=0.1, g=0.0001 
 train_err :46.33237133237134	 val_err:46.32034632034632

C=0.1, g=0.001 
 train_err :22.619047619047617	 val_err:21.693121693121697

C=0.1, g=0.01 
 train_err :22.643097643097644	 val_err:21.645021645021643

C=0.1, g=0.1 
 train_err :22.4025974025974	 val_err:21.789321789321793

C=0.1, g=1 
 train_err :27.441077441077443	 val_err:31.89033189033189

C=1, g=0.0001 
 train_err :22.619047619047617	 val_err:21.693121693121697

C=1, g=0.001 
 train_err :22.619047619047617	 val_err:21.693121693121697

C=1, g=0.01 
 train_err :22.619047619047617	 val_err:21.693121693121697

C=1, g=0.1 
 train_err :21.42857142857143	 val_err:21.212121212121215

C=1, g=1 
 train_err :14.03318903318903	 val_err:23.23232323232324

C=10, g=0.0001 
 train_err :22.619047619047617	 val_err:21.693121693121697

C=10, g=0.001 
 train_err :22.619047619047617	 val_err:21.693121693121697

C=10, g=0.01 
 train_err :22.59499759499759	 val_err:21.693121693121697

C=10, g=0.1 
 train_err :18.458393458393463	 v

In [131]:
for i in range(len(train_errs1)):
    if (((i+1)%5==0 & (i!=0))):
        print(f"{round(train_errs1[i],7)}",end ="\n")
    else:
        print(f"{round(train_errs1[i],7)}",end ="\t")
print()
for i in range(len(val_errs1)):
    if (((i+1)%5==0 & (i!=0))):
        print(f"{round(val_errs1[i],7)}",end ="\n")
    else:
        print(f"{round(val_errs1[i],7)}",end ="\t")

46.3323713	22.6190476	22.6430976	22.4025974	27.4410774
22.6190476	22.6190476	22.6190476	21.4285714	14.033189
22.6190476	22.6190476	22.5949976	18.4583935	5.3391053
22.6190476	22.6190476	22.4867725	17.2919673	3.6556037
22.6190476	22.6190476	22.2582973	15.7287157	2.0683021
22.6190476	22.6190476	21.8614719	14.2736893	1.4069264

46.3203463	21.6931217	21.6450216	21.7893218	31.8903319
21.6931217	21.6931217	21.6931217	21.2121212	23.2323232
21.6931217	21.6931217	21.6931217	21.5488215	26.9360269
21.6931217	21.6931217	21.7412217	22.1260221	27.5132275
21.6931217	21.6931217	21.5488215	23.6171236	28.6676287
21.6931217	21.6931217	21.8374218	23.4247234	29.2929293


In [132]:
#điểm của mô hình mlpclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_val
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train_v, y_train_v)
y_pred = mlpclf_model.predict(x_val)
print(classification_report(y_val, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

              precision    recall  f1-score   support

      norain       0.80      0.83      0.81      1116
        rain       0.79      0.76      0.78       963

    accuracy                           0.80      2079
   macro avg       0.80      0.79      0.79      2079
weighted avg       0.80      0.80      0.80      2079



C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [133]:
#điểm của mô hình svmclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_train_v
svmclf_model.set_params(svm__C=best_C,svm__gamma=best_gamma)
svmclf_model.fit(x_train_v,y_train_v)
y_pred = svmclf_model.predict(x_val)

print(classification_report(y_val, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

              precision    recall  f1-score   support

      norain       0.79      0.82      0.81      1116
        rain       0.78      0.75      0.77       963

    accuracy                           0.79      2079
   macro avg       0.79      0.79      0.79      2079
weighted avg       0.79      0.79      0.79      2079



# NHẬN XÉT
- 2 mô hình khá cân băng
- f1 score của SVM của nhãn rain cao hơn của MLP 0.1
- MLP tốt hơn SVM 1 chút trong việc dự đoán ra nhãn "rain" nên chọn MLP

In [134]:
#huấn luyện trên toàn tập train
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train,y_train)
y_pred = svmclf_model.predict(x_train)

print(classification_report(y_train, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

              precision    recall  f1-score   support

      norain       0.79      0.82      0.80      5579
        rain       0.78      0.75      0.76      4816

    accuracy                           0.79     10395
   macro avg       0.79      0.78      0.78     10395
weighted avg       0.79      0.79      0.79     10395



In [135]:
# Điểm mô hình trên test set
y_pred = mlpclf_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      norain       0.77      0.83      0.80       620
        rain       0.78      0.72      0.75       536

    accuracy                           0.78      1156
   macro avg       0.78      0.77      0.77      1156
weighted avg       0.78      0.78      0.78      1156



C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_19664\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

# Nhận xét
- Độ chính xác của mô mình đạt được mong đợi (>80%)
- Mô hình dự đoán tốt đối với nhãn không mưa (f1 : 0.91)
- Mô hình dự đoán nhãn mưa không tốt lắm, nhưng cũng không hề tệ tuy nhiên độ hiệu quả cũng tương đối
    -Đưa ra được 65% trường hợp mưa với độ chính xác 78%

# Kết luận
- Dự đoán là khi áp dụng vào thực tế, mô hình hoạt động sẽ không tốt, nhưng không quá tệ
- Có thể sử dụng làm nguồn tham khảo nếu không thể xem dự báo thời tiết vì một lý do nào đó 